In [1]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.7
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [2]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("kem_test")
         .getOrCreate())

In [3]:
spark

In [4]:
!hdfs dfs -ls /labs/laba01/ml-100k

Found 23 items
-rw-r--r--   3 hdfs hdfs       6750 2022-01-06 18:46 /labs/laba01/ml-100k/README
-rw-r--r--   3 hdfs hdfs        716 2022-01-06 18:46 /labs/laba01/ml-100k/allbut.pl
-rw-r--r--   3 hdfs hdfs        643 2022-01-06 18:46 /labs/laba01/ml-100k/mku.sh
-rw-r--r--   3 hdfs hdfs    1979173 2022-01-06 18:46 /labs/laba01/ml-100k/u.data
-rw-r--r--   3 hdfs hdfs        202 2022-01-06 18:46 /labs/laba01/ml-100k/u.genre
-rw-r--r--   3 hdfs hdfs         36 2022-01-06 18:46 /labs/laba01/ml-100k/u.info
-rw-r--r--   3 hdfs hdfs     236344 2022-01-06 18:46 /labs/laba01/ml-100k/u.item
-rw-r--r--   3 hdfs hdfs        193 2022-01-06 18:46 /labs/laba01/ml-100k/u.occupation
-rw-r--r--   3 hdfs hdfs      22628 2022-01-06 18:46 /labs/laba01/ml-100k/u.user
-rw-r--r--   3 hdfs hdfs    1586544 2022-01-06 18:46 /labs/laba01/ml-100k/u1.base
-rw-r--r--   3 hdfs hdfs     392629 2022-01-06 18:46 /labs/laba01/ml-100k/u1.test
-rw-r--r--   3 hdfs hdfs    1583948 2022-01-06 18:46 /labs/laba01/ml-100k/u2.base


In [5]:
rdd2_scores = spark.sparkContext.textFile("/labs/laba01/ml-100k/u.data")

In [6]:
rdd2_scores = rdd2_scores.map(lambda x: x.split("\t"))

In [7]:
rdd2_scores.take(5)

[['196', '242', '3', '881250949'],
 ['186', '302', '3', '891717742'],
 ['22', '377', '1', '878887116'],
 ['244', '51', '2', '880606923'],
 ['166', '346', '1', '886397596']]

In [8]:
v_film = rdd2_scores.filter(lambda x: x[1] == '22')

In [9]:
v_film.take(5)

[['269', '22', '1', '891448072'],
 ['8', '22', '5', '879362183'],
 ['90', '22', '4', '891384357'],
 ['222', '22', '5', '878183285'],
 ['313', '22', '3', '891014870']]

In [ ]:
#rdd2_films = spark.sparkContext.textFile("/labs/laba01/ml-100k/u.item")

In [ ]:
#rdd2_films = rdd2_films.map(lambda x: x.split("|"))

In [ ]:
#rdd2_films.take(5)

In [10]:
v_film_score = (v_film.map(lambda x: (x[2],1))
                 .countByKey()
                 .items())

In [11]:
all_score = (rdd2_scores.map(lambda x: (x[2],1))
                 .countByKey()
                 .items())

In [12]:
sorted(v_film_score)

[('1', 5), ('2', 14), ('3', 46), ('4', 98), ('5', 134)]

In [13]:
sorted(all_score)

[('1', 6110), ('2', 11370), ('3', 27145), ('4', 34174), ('5', 21201)]

In [14]:
res_lr1 = {'hist_film':[item[1] for item in sorted(v_film_score)],
          'hist_all':[item[1] for item in sorted(all_score)]}

In [15]:
with open('lab01.json', 'w') as file:
    json.dump(res_lr1, file)

In [16]:
spark.stop()